In [1]:
#Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
#import gpflow
#import tensorflow as tf
#from gpflow import settings 
import GPy
np.random.seed(1)
%matplotlib inline

In [4]:
#Methods
#Useful methods
def get_elems_list(row, cols):
    elem_comb = {}
    elem_list = []
    i = 0
    for a in row:
        if not math.isnan(a):
            elem_list.append(cols[i])
        i+=1
    elem_comb['elements'] = elem_list
    return elem_comb

def get_data_rep(df):
    elements = get_elements(df)
    ecols = elements.columns
    data = df.drop(columns = ecols)
    e = pd.DataFrame()
    for i in range (0, len(elements)):
        e =e.append(get_elems_list(elements.loc[i], ecols), ignore_index=True)
    print(e)
    data['elements'] = e['elements']
    return data

def get_oth_props(df):
    elements = get_elements(df)
    ecols = elements.columns
    data = df.drop(columns = ecols)
    return data


def run_on_df(df, f):
    a = pd.DataFrame()
    for c in df.columns:
        a[c] = f(df[c],0)
    return a

def get_heatmap(df):
    return df.style.background_gradient(cmap='magma') 

def get_corrs(df, func, ind):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(func(df[r], df[c])[ind], 4)

    return pvalues

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

def notnulls(data, col):
    return data[data[col].notnull()][col]

def numerify_df(data):
    for c in data.columns:
        try:
            data[c] = data[c].astype(float)
        except:
            print(c)
    return data

def drop_txt_rows(data):
    for col in data.columns:
        if data[col].dtype ==object:
            data = data.drop(col, axis=1)  
    data = data.fillna(0)
    return data

def get_elements(df):
    new_cols = []
    for c in df.columns:
        if len(c)<3:
            new_cols.append(c)
            
    return df[new_cols]

            

In [5]:
#Getting data
df = pd.read_csv("final_numeric_data.csv").drop(columns = {'Unnamed: 0'})
columns = list(df.columns)
elems = get_elements(df)
#elems = numerify_df(elems).astype(float).to_numpy()
#elems[elems==0] = np.nan
df = numerify_df(df).astype(float).to_numpy()
df[df==0] = np.nan

In [6]:
def get_data(df,prop):
    tmp = np.concatenate([elems, df[:,columns.index(prop)].reshape(3255,1)], axis=1)
    tmp = np.nan_to_num(tmp)
    #Remove these 2 next lines to include 0 values
    tmp[tmp==0] = np.nan
    tmp = tmp[pd.notnull(tmp[:,-1])]
    y = tmp[:,-1]
    X = np.nan_to_num(tmp)
    X = X[:,:-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)
    y_train = y_train.reshape(y_train.shape[0], 1)
    y_test = y_test.reshape(y_test.shape[0], 1)
    return X_train, X_test, y_train, y_test

In [11]:
def get_err(prop, kern):
    data = get_data(df,prop)
    #Basic regression model
    k1 = kern(input_dim=data[0].shape[1])
    k2 = GPy.kern.WhiteHeteroscedastic(input_dim=data[0].shape[1])
    ker = k1+k2
    m = GPy.models.GPRegression(data[0], data[2], ker)
    m.optimize()
    pred = m.predict(data[1])
    return mean_squared_error(pred[0], data[3])

In [8]:
zero = data[0]<1

NameError: name 'data' is not defined

In [81]:
np.count_nonzero(zero)

20138

In [82]:
zero = data[3]<1

In [83]:
np.count_nonzero(zero)

0

In [9]:
def try_kern(kern):
    a = []
    for i in range (0,16):
        try:
            err = get_err(columns[i], kern)
            print(columns[i]+ " - "+str(err))
        except:
            print(columns[i] +" DID NOT WORK")
    return a

In [12]:
try_kern(GPy.kern.RBF)


Category DID NOT WORK
Charpy DID NOT WORK
DIN_W DID NOT WORK
Elongation DID NOT WORK
HT_Temp DID NOT WORK
Hardness_Scale DID NOT WORK
IE-20 DID NOT WORK
Manual metal arc_Heat input DID NOT WORK
Manual metal arc_Interpass temperature DID NOT WORK
Redry_Time DID NOT WORK
Reduction of area DID NOT WORK
Rp0 DID NOT WORK
Tensile DID NOT WORK
Ultimate tensile strength DID NOT WORK
Yield DID NOT WORK
Yield strength DID NOT WORK


[]

In [ ]:
#ntks, neural tangent kernels
#yasmaan bahari
#lmfit
#.7, .2, .1
#sheet for each property - variations, performances - test, validation, test ()
#RMSE
#Sklearn look at other libraries - Knife based regression, shallow perceptrons, bayesian 
#Looking at imagery data
#Recent microsoft paper
#Reviews of modern physics - info theory, particle physics
#Graph networks